In [1]:
import sys
sys.path.insert(0, './..')
sys.path.insert(0, '../data')

import matplotlib.pyplot as plt
import numpy as np
import plots as pl
from models import model
import torch
from utils import dev, load_data, classification
from torchvision import datasets, transforms
from models import eval
import dill
from robustness1 import model_utils, datasets, train, defaults
from robustness1.datasets import CIFAR
from torch.utils.data import DataLoader

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

/mnt/qb/home/bethge/dschultheiss/AdversarialDecomposition/robustness1/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


# Load CIFAR model

# Evaluate model

In [14]:
ds = CIFAR('../data/')
m_nat, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/nat_diff.pt')
m_rob, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/rob_diff.pt')
train_loader, val_loader = ds.make_loaders(batch_size=128, workers=8)

# # Create a cox store for logging
# out_store = cox.store.Store('./nat')

# # Hard-coded base parameters
# train_kwargs = {
#     'out_dir': "./rob/train_out",
#     'eval_only': 1,
#     'adv_eval': 1,
#     'constraint': '2',
#     'eps': 0.5,
#     'attack_lr': 1.5,
#     'attack_steps': 20
# }
# train_args = Parameters(train_kwargs)

# # Fill whatever parameters are missing from the defaults
# train_args = defaults.check_and_fill_args(train_args, defaults.CONFIG_ARGS, CIFAR)
# train_args = defaults.check_and_fill_args(train_args, defaults.PGD_ARGS, CIFAR)


# # Train a model
# train.eval_model(train_args, m_rob, val_loader, store=out_store)


# Create a cox store for logging
out_store = cox.store.Store('./rob')

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "./rob/train_out",
    'eval_only': 1,
    'adv_eval': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 1.5,
    'attack_steps': 20
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args, defaults.CONFIG_ARGS, CIFAR)
train_args = defaults.check_and_fill_args(train_args, defaults.PGD_ARGS, CIFAR)

# Train a model
train.eval_model(train_args, m_rob, val_loader, store=out_store)




Custom model
=> loading checkpoint './models/cifar_models/nat_diff.pt'
=> loaded checkpoint './models/cifar_models/nat_diff.pt' (epoch 111)
Custom model
=> loading checkpoint './models/cifar_models/rob_diff.pt'
=> loaded checkpoint './models/cifar_models/rob_diff.pt' (epoch 150)
==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified
Logging in: /mnt/qb/home/bethge/dschultheiss/AdversarialDecomposition/rob/05701b23-7dfe-478a-b339-997a93556188


Val Epoch:0 | Loss 0.4375 | NatPrec1 85.430 | NatPrec5 99.410 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:03<00:00, 25.98it/s]
Val Epoch:0 | Loss 0.9288 | AdvPrec1 65.070 | AdvPrec5 98.180 | Reg term: 0.0 ||: 100%|██████████| 79/79 [02:18<00:00,  1.75s/it]


{'epoch': 0,
 'nat_prec1': tensor(85.4300, device='cuda:0'),
 'adv_prec1': tensor(65.0700, device='cuda:0'),
 'nat_loss': 0.4375362557411194,
 'adv_loss': 0.9287609119415283,
 'train_prec1': nan,
 'train_loss': nan,
 'time': 142.0380403995514}

# Make evenly distributed dataset with true samples only

In [20]:
ds = CIFAR('../data/')
m_nat, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/nat_diff.pt')
m_nat.to(dev())
m_nat.double()
m_nat.eval()

m_rob, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/rob_diff.pt')
m_rob.to(dev())
m_rob.double()
m_rob.eval()


images, labels = load_data(1000, random=False, train=False, d_set='CIFAR')
image_batches = torch.split(images, 128)
label_batches = torch.split(labels, 128)

correct_classes = np.full(len(images), True)
preds = np.empty(0)

for img, labels_ in zip(image_batches, label_batches):
    preds = np.concatenate((preds, m_nat(img.to(dev()))[0].argmax(-1).cpu().numpy()), axis=0)
    
correct_classes[~(preds==labels.numpy())]=False

preds = np.empty(0)
for img, labels_ in zip(image_batches, label_batches):
    preds = np.concatenate((preds, m_rob(img.to(dev()))[0].argmax(-1).cpu().numpy()), axis=0)
    
correct_classes[~(preds==labels.numpy())]=False

images = images[correct_classes]
labels = labels[correct_classes]

data = {
    'images': images.cpu(),
    'labels': labels.cpu()
}

save_path = '/home/bethge/dschultheiss/AdversarialDecomposition/data/CIFAR/stable_data_diff.npy'
np.save(save_path, data)
print('Accuracy: ' + str(100*correct/all_n))

Custom model
=> loading checkpoint './models/cifar_models/nat_diff.pt'
=> loaded checkpoint './models/cifar_models/nat_diff.pt' (epoch 111)
Custom model
=> loading checkpoint './models/cifar_models/rob_diff.pt'
=> loaded checkpoint './models/cifar_models/rob_diff.pt' (epoch 150)
Files already downloaded and verified
Accuracy: 85.0


# Concatenate data

In [2]:
model_id = 0
n_batches = 100
model_type = "natural"

data = np.load('./data/cifar_' + model_type + '_0.npy', allow_pickle=True).item()
advs = data['advs']
classes = data['adv_class']
pert_lengths = data['pert_lengths']
dirs = data['dirs']
images = data['images']
labels = data['labels']


for batch_n in np.arange(1,n_batches):
    path = './data/cifar_' + model_type + '_'+str(batch_n)+'.npy'
    data = np.load(path, allow_pickle=True).item()
    advs = np.concatenate((advs, data['advs']), axis=0)
    classes = np.concatenate((classes, data['adv_class']), axis=0)
    pert_lengths = np.concatenate((pert_lengths, data['pert_lengths']), axis=0)
    dirs = np.concatenate((dirs, data['dirs']), axis=0)
    images = np.concatenate((images, data['images']), axis=0)
    labels = np.concatenate((labels, data['labels']), axis=0)

In [3]:
pert_lengths[pert_lengths==0] = np.nan
sort_idx = np.argsort(pert_lengths,axis=-1)
for i, idx in enumerate(sort_idx):
    pert_lengths[i] = pert_lengths[i,idx]
    dirs[i] = dirs[i,idx]
    advs[i] = advs[i,idx]
    classes[i] = classes[i,idx]

In [4]:
np.isnan(pert_lengths).sum()

0

In [5]:
data = {
    'advs': advs,
    'dirs': dirs,
    'adv_class': classes,
    'pert_lengths': pert_lengths,
    'images': images,
    'labels': labels,
}
save_path = '/home/bethge/dschultheiss/AdversarialDecomposition/data/cifar_' + model_type + '_diff'
np.save(save_path, data)